# 三者对比
![](../images/Python并发编程/三者对比.png)

# 三者选择
![](../images/Python并发编程/三者选择.png)

# 全局解释器锁
## Python速度慢的两大原因
1. Python是动态类型语言，边解释边执行
2. GIL无法利用多核CPU并发执行
## GIL是什么
全局解释器锁是编程语言解释器用于同步线程的一种机制，它使得任何时刻只有一个线程在执行

即便在多核心处理器上，使用GIL的解释器也只允许同一时间执行一个线程
![](../images/Python并发编程/GIL.png)

## 为什么会有GIL？
在一个古老的单核CPU上调度多个线程任务，大家相互共享一个全局锁，谁在CPU执行，谁就占有这把锁，直到这个线程因为IO操作或者Timer Tick到期让出CPU，没有在执行的线程就安静的等待着这把锁。使用全局解释器锁的方案是最简单的

由于多个线程同时对数据进行操作，会引发数据不一致，导致内存泄漏，我们可以对其进行加锁，所以Cpython就创建了GIL锁
但是既然有了锁，一个对象就需要一把锁，那么多个对象就会有多把锁，可能会给我们带来2个问题

1. 死锁（线程之间互相争抢锁的资源）
2. 反复获取和释放锁而导致性能降低

为了保证单线程情况下python的正常执行和效率，GIL锁（单一锁）由此产生了，它添加了一个规则，即任何Python字节码的执行都需要获取解释器锁。这样可以防止死锁（因为只有一个锁），并且不会带来太多的性能开销。但这实际上使所有受CPU约束的Python程序（指的是CPU密集型程序）都是单线程的。

### 另一个思路：为什么 JVM 不需要 GIL？
不妨思考这样一个问题：为什么 JVM 不需要 GIL ？试想一下，你费尽九牛二虎之力写了一个 Hash Map，准备丢给别人用，突然别人告诉你我会多线程同时访问你这个东西，此时你有两个选择：

改造底层设计，例如给哈希空间中的每一个桶都加上锁，让整个 Hash Map 能在大部分情境下真正支持多线程并发；当然如果你这个 Map 带缩扩容之类的，那要改造成线程安全的就更麻烦了。

直接给整个 Map 加一个全局锁，谁想访问就得先拿这个锁，一分钟改完。这里的 Map 就是 CPython，全局锁就是 GIL，只是复杂程度要高很多。

CPython 在两种策略中选择了后者（这是 “CPython 的内存管理为什么不是线程安全的“ 的回答，选择如此），因此同一时刻永远只有一个线程在真正执行；而 JVM 在两种策略中选择了前者，使得它真正支持多核并发。

但是后者并非除了省事之外就一无是处。选择后者，CPython 运行起来像是这样的：

```
取得GIL(); 
...
...
...
释放GIL();
```

选择前者，CPython 运行起来就像这样：

```
取得某个锁();
...
释放某个锁();
...
...
取得某个锁();
...
释放某个锁();
```

所以，对于解释型语言，支持线程多核并行是有代价的，就是单个线程效率的下降。一旦底层解释器采用了前者那样的设计，用更多小粒度的锁来替代一个大锁，那么即便是单线程程序，也会不可避免地在运行过程中有许多取得和释放锁的操作，会让效率大打折扣。

线程效率不仅仅对单线程程序重要，比如，一些 IO 密集型的程序，看起来有很多线程，但大部分线程大多数时间都在挂机，实际疯狂干活的可能就一两个线程，这种场景下，单位线程效率也是非常重要的，即便用上 2 个核一起跑两个线程，单位线程效率下降多了，也未必快得了多少。

总结一下，主要是出于历史原因和单位线程效率的考虑， CPython 至今还有 GIL

## 有GIL一定线程安全吗？
有GIL并不意味着python一定是线程安全的，那什么时候安全，什么时候不安全，我们必须搞清楚。之前我们已经说过，一个线程有两种情况下会释放全局解释器锁，一种情况是在该线程进入IO操作之前，会主动释放GIL。另一种情况是解释器不间断运行了1000字节码（Py2）或运行15毫秒（Py3）后，该线程也会放弃GIL。既然一个线程可能随时会失去GIL，那么这就一定会涉及到线程安全的问题。GIL虽然从设计的出发点就是考虑到线程安全，但这种线程安全是粗粒度的线程安全，即不需要程序员自己对线程进行加锁处理（同理，所谓细粒度就是指程序员需要自行加、解锁来保证线程安全，典型代表是 Java , 而 CPthon 中是粗粒度的锁，即语言层面本身维护着一个全局的锁机制,用来保证线程安全）。那么什么时候需要加锁，什么时候不需要加锁，这个需要具体情况具体分析。下面我们就来针对每种可能的情况进行分析和总结。

首先来看第一种线程释放GIL的情况。假设现在线程A因为进入IO操作而主动释放了GIL，那么在这种情况下，由于线程A的IO操作等待时间不确定，那么等待的线程B一定会得到GIL锁，这种比较“礼貌的”情况我们一般称为“协同式多任务处理”，相当于大家按照协商好的规则来，线程是安全的，不需要额外加锁。

接下来，我们来看另外一种情况，即线程A是因为解释器不间断执行了1000字节码的指令或不间断运行了15毫秒而放弃了GIL，那么此时实际上线程A和线程B将同时竞争GIL锁。在同时竞争的情况下，实际上谁会竞争成功是不确定的一个结果，所以一般被称为“抢占式多任务处理”，这种情况下当然就看谁抢得厉害了。当然，在python3上由于对GIL做了优化，并且会动态调整线程的优先级，所以线程B的优先级会比较高，但仍然无法肯定线程B就一定会拿到GIL。那么在这种情况下，线程可能就会出现不安全的状态。针对这种纯计算的操作，我们用一段代码来演示下这种线程不安全的状态。代码如下：


In [28]:
import threading

n = 0

def add():
    global n
    for i in range(1000000):
        n = n + 1
def sub():
    global n
    for i in range(1000000):
        n = n - 1

if __name__ == "__main__":
    t1 = threading.Thread(target=add)
    t2 = threading.Thread(target=sub)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("n的值为:", n)


n的值为: 447099


上面的代码很简单，分别用线程1和线程2对全局变量n进行了1000000次的加和减操作。如果线程安全的话，那么最终的结果n应该还是为0。但实际上，我们运行之后，会发现这个n的值有时大有时小，完全不确定。这就是典型的多个线程操作同一个全局变量造成的线程不安全的问题。

在线程中，我们主要是执行了一个加法和减法的操作。为了方便说明问题，我们把函数最简化到一个加法函数和一个减法函数，来分析它们的字节码执行过程，来看看释放GIL锁是怎么引起这个问题的。演示代码如下：

In [29]:
import dis
n = 0

def add():
    global n
    n = n + 1

print(dis.dis(add))

def sub():
    global n
    n = n - 1
print(dis.dis(sub))

  6           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None
 12           0 LOAD_GLOBAL              0 (n)
              2 LOAD_CONST               1 (1)
              4 BINARY_SUBTRACT
              6 STORE_GLOBAL             0 (n)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None


dis模块中的dis方法可以打印出一个函数对应的字节码执行过程，所以非常方便我们进行分析。运行结果如上：

不管是加法还是减法运算，都会分为4步完成。以加法为例，第一步是LOAD_GLOBAL(加载全局变量n)，第二步LOAD_CONST(加载常量1)，第三步进行二进制的加法，第四步将计算结果存储到全局变量n中，加法计算结束。这四个指令如果能够保证被作为一个整体完整地运行，那么是不会产生问题的，但根据前面说的线程释放GIL的原则，那么很有可能在线程正在执行这四步中的任何一步的时候释放掉GIL而进入等待状态，这个时候发生的事情就比较有意思了。为了方便大家理解，我拿一种比较极端的情况来说明一下。比如我们在加法运算中，正准备执行第四步的时候，很不幸失去了GIL，进入等待状态（注意此时n值仍然为0）。减法运算的线程开始执行，它加载了全局变量n（值为0），并进行减法相关的计算，它也在执行第三步的时候失去了GIL，此时它进入等待状态，加法运算继续。上一次加法计算继续运行第4步，即把加法运算结果赋值给全局变量n，那么此时n的值为1。同样道理，减法操作拿回GIL时，它之前已经加载了为0的n的值，所以它继续操作到最后赋值那步时，n的值就为0-1=-1。换句话说，n的值要么为1，要么为-1，但我们期望的应该是0。这就造成了线程不安全的情形。最终，经过百万次这样不确定的加减操作，那么结果一定是不确定的。这就是引起这个问题的过程和原因。

接下来，我们还要解决另外一个问题，也就是既然GIL从粗粒度情况下存在线程不安全的可能性，那么是不是所有非IO操作引起的GIL释放都要加锁来解决线程安全的问题。这个问题同样要分情况，因为python跟其他线程自由的语言比如 Java相比，它有很多操作是原子级的，针对原子级的操作，由于方法本身是单个字节码，所以线程没有办法在调用期间放弃GIL。典型的例子比如sort方法，我们同样可以看看这种原子级的操作在python的字节码中是什么样子，代码演示如下：

In [30]:
import dis

lst = [4, 1, 3, 2]

def foo():
    lst.sort()

print(dis.dis(foo))

  6           0 LOAD_GLOBAL              0 (lst)
              2 LOAD_METHOD              1 (sort)
              4 CALL_METHOD              0
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
None


从字节码的角度，调用sort操作是原子级无法再分的，所以线程不会在执行期间发生GIL释放的情况，也就是说我们可以认为sort操作是线程安全的，不需要加锁。而我们上面演示的加法和减法操作则不是原子级的，所以我们必须要加锁才能保证线程安全。

所以，总结一下，如果多线程的操作中不是IO密集型，并且计算操作不是原子级的操作时，那么我们需要考虑线程安全问题，否则都不需要考虑线程安全。当然，为了避免担心哪个操作是原子的，我们可以遵循一个简单的原则：始终围绕共享可变状态的读取和写入加锁。毕竟，在 Python 中获取一个 threading.Lock 也就是一行代码的事。

## 怎样规避GIL带来的限制
![](../images/Python并发编程/规避GIL.png)

## Python创建多线程的方法

In [ ]:
# 1. 准备一个函数
def my_func(a,b):
    do_craw(a,b)    # 执行爬虫方法
# 2. 怎样创建一个线程
import threading
t = threading.Thread(target=my_func,args=(100,200))
# 3. 启动线程
t.start()
# 4. 等待结束
t.join()

In [15]:
import requests
urls = [
    f'https://www.cnblogs.com/#{page}' for page in range(1,50+1)
]
def craw(url):  # 爬虫代码
    r=requests.get(url)
    print(url,len(r.text))
craw(urls[0])

https://www.cnblogs.com/#1 71656


In [8]:
import threading

def single_thread():    # 单线程爬虫
    print('single thread begin')
    for url in urls:
        craw(url)
    print('single thread end')
def multi_thread():   # 多线程爬虫
    print('multi thread begin')
    threads = []
    for url in urls:
        threads.append(
            threading.Thread(target=craw,args=(url,))
        )
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print('multi thread end')

In [16]:
import time
start = time.time()
single_thread()
end = time.time()
print('single thread cost:',end-start,"seconds")


single thread begin
https://www.cnblogs.com/#1 71656
https://www.cnblogs.com/#2 71656
https://www.cnblogs.com/#3 71656
https://www.cnblogs.com/#4 71656
https://www.cnblogs.com/#5 71656
https://www.cnblogs.com/#6 71656
https://www.cnblogs.com/#7 71656
https://www.cnblogs.com/#8 71656
https://www.cnblogs.com/#9 71656
https://www.cnblogs.com/#10 71656
https://www.cnblogs.com/#11 71656
https://www.cnblogs.com/#12 71656
https://www.cnblogs.com/#13 71656
https://www.cnblogs.com/#14 71656
https://www.cnblogs.com/#15 71656
https://www.cnblogs.com/#16 71656
https://www.cnblogs.com/#17 71656
https://www.cnblogs.com/#18 71656
https://www.cnblogs.com/#19 71656
https://www.cnblogs.com/#20 71656
https://www.cnblogs.com/#21 71656
https://www.cnblogs.com/#22 71656
https://www.cnblogs.com/#23 71656
https://www.cnblogs.com/#24 71656
https://www.cnblogs.com/#25 71656
https://www.cnblogs.com/#26 71656
https://www.cnblogs.com/#27 71656
https://www.cnblogs.com/#28 71656
https://www.cnblogs.com/#29 71656
htt

In [17]:
start = time.time()
multi_thread()
end = time.time()
print('single thread cost:',end-start,"seconds")

multi thread begin
https://www.cnblogs.com/#8 71656
https://www.cnblogs.com/#4 71656
https://www.cnblogs.com/#3 71656
https://www.cnblogs.com/#12 71656
https://www.cnblogs.com/#14 71656
https://www.cnblogs.com/#6 71656
https://www.cnblogs.com/#9 71656
https://www.cnblogs.com/#13 71656
https://www.cnblogs.com/#20 71656
https://www.cnblogs.com/#16 71656
https://www.cnblogs.com/#11 71656
https://www.cnblogs.com/#10 71656
https://www.cnblogs.com/#18 71656
https://www.cnblogs.com/#5 71656
https://www.cnblogs.com/#19 71656
https://www.cnblogs.com/#1 71656
https://www.cnblogs.com/#21 71656
https://www.cnblogs.com/#22 71656
https://www.cnblogs.com/#23 71656
https://www.cnblogs.com/#25 71656
https://www.cnblogs.com/#27 71656
https://www.cnblogs.com/#26 71656
https://www.cnblogs.com/#32 71656
https://www.cnblogs.com/#17 71656
https://www.cnblogs.com/#34 71656
https://www.cnblogs.com/#2 71656
https://www.cnblogs.com/#7 71656
https://www.cnblogs.com/#36 71656
https://www.cnblogs.com/#30 71656
http

## 多组件的Pipeline技术架构
复杂的事情一般不会一下子做完，而是会分很多中间步骤一步步完成
## 生产者消费者爬虫的架构
![](../images/Python并发编程/爬虫架构.png)

## 多线程数据通信的queue.Queue
可以用于多线程之间的，线程安全（并发同时访问数据不会出现数据冲突）的数据通信

1. 导入类库
import queue
2. 创建Queue
q = queue.Queue()
3. 添加元素(阻塞的)
q.put(item)
4. 获取元素(阻塞的)
item = q.get()
5. 查询状态
    * 查看元素的多少
    q.qsize()
    * 判断是否为空
    q.empty()
    * 判断是否已满
    q.full()

In [19]:
from bs4 import BeautifulSoup
def craw(url):
    r=requests.get(url)
    return r.text

def parse(html):
    # class="post-item-title"
    soup = BeautifulSoup(html,"html.parser")
    links = soup.find_all("a",class_="post-item-title")
    return [(link["href"],link.get_text()) for link in links]

parse(craw(urls[0]))

[('https://www.cnblogs.com/xiaowange/p/18146721', '学习 XQuery：XML数据查询的关键'),
 ('https://www.cnblogs.com/cnb-yuchen/p/18146625', '如何使用XSSFWorkbook读取文本薄？'),
 ('https://www.cnblogs.com/gmmy/p/18146496', '使用纯c#在本地部署多模态模型，让本地模型也可以理解图像'),
 ('https://www.cnblogs.com/hosseini/p/18146328', '单例模式'),
 ('https://www.cnblogs.com/timefiles/p/18146188', 'ImageJ软件使用教程（三）：目标计数'),
 ('https://www.cnblogs.com/gggod/p/18145831', 'ddddocr基本使用和介绍'),
 ('https://www.cnblogs.com/hk416hasu/p/18145573', 'OS如何保持对计算机的控制权？'),
 ('https://www.cnblogs.com/fkxxgis/p/18145575',
  '修改中文、英文参考文献在文末列表中的顺序：EndNote'),
 ('https://www.cnblogs.com/VincentLee/p/18145569',
  'SURE：增强不确定性估计的组合拳，快加入到你的训练指南吧 | CVPR 2024'),
 ('https://www.cnblogs.com/ting1/p/18145360',
  'NL2SQL技术方案系列(1)：NL2API、NL2SQL技术路径选择；LLM选型与Prompt工程技巧，揭秘项目落地优化之道'),
 ('https://www.cnblogs.com/Vsinger-LuoTianYi/p/18136248',
  '【学习笔记】 字符串基础 ： 后缀自动机(基础篇)'),
 ('https://www.cnblogs.com/isharetech/p/18145338', '深度解读《深度探索C++对象模型》之返回值优化'),
 ('https://www.cnblogs.com/mingupu

## 生产者消费者版本的爬虫

In [25]:
import queue
import random
def do_craw(url_queue:queue.Queue,html_queue:queue.Queue):
    while True:
        url = url_queue.get()
        html = craw(url)
        html_queue.put(html)
        print(threading.current_thread().name,f"craw {url}","url_queue.size=",url_queue.qsize())
        time.sleep(random.randint(1,2))

def do_parse(html_queue: queue.Queue,fout):
    while True:
        html = html_queue.get()
        results = parse(html)
        for result in results:
            fout.write(str(result)+'\n')
        print(threading.current_thread().name,f"result.size",len(results),"html_queue.size=",html_queue.qsize())
        time.sleep(random.randint(1,2))

url_queue = queue.Queue()
html_queue = queue.Queue()
for url in urls:
    url_queue.put(url)
for idx in range(3):    # 三个线程下载网页
    t=threading.Thread(target=do_craw, args=(url_queue,html_queue),name=f"craw{idx}")
    t.start()

fout = open('02.data.txt','w')
for idx in range(2):    # 两个线程解析网页并存储
    t=threading.Thread(target=do_parse, args=(html_queue,fout),name=f"parse{idx}")
    t.start()


craw1 craw https://www.cnblogs.com/#2 url_queue.size= 47
parse0 result.size 20 html_queue.size= 0
craw2 craw https://www.cnblogs.com/#3 url_queue.size= 47
craw0 craw https://www.cnblogs.com/#1 url_queue.size= 47
parse1 result.size 20 html_queue.size= 1
parse0 result.size 20 html_queue.size= 0
craw1 craw https://www.cnblogs.com/#4 url_queue.size= 46
parse1 result.size 20 html_queue.size= 0
craw2 craw https://www.cnblogs.com/#5 url_queue.size= 44
craw0 craw https://www.cnblogs.com/#6 url_queue.size= 44
parse1 result.size 20 html_queue.size= 1
parse0 result.size 20 html_queue.size= 0
craw2 craw https://www.cnblogs.com/#7 url_queue.size= 43
craw1 craw https://www.cnblogs.com/#8 url_queue.size= 42
parse0 result.size 20 html_queue.size= 1
craw0 craw https://www.cnblogs.com/#9 url_queue.size= 40
parse1 result.size 20 html_queue.size= 1
craw2 craw https://www.cnblogs.com/#10 url_queue.size= 39
craw1 craw https://www.cnblogs.com/#11 url_queue.size= 39
parse0 result.size 20 html_queue.size= 2
pa

## 线程安全概念介绍
线程安全指的是某个函数、函数库在多线程环境中被调用时，能够正确的处理多个线程之间的共享变量，使得程序功能正确完成

由于线程的执行随时会发生切换，就造成了不可预料的结果，出现线程不安全